In [1]:
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split


import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
import matplotlib
import matplotlib.pyplot as pyplot # for plotting


import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier,Pool
from IPython.display import display
import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import norm
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import time
import glob
import sys
import os
import gc

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train = pd.read_csv('train_modified.csv')
test = pd.read_csv('test_modified.csv')
train['target']=train['Disbursed']
train=train.drop('Disbursed',axis=1)
target = 'Disbursed'
IDcol = 'ID'

In [3]:
data=train.append(test)
data.head(2)

,Age,Device_Type_0,Device_Type_1,EMI_Loan_Submitted_Missing,Existing_EMI,Filled_Form_0,Filled_Form_1,Gender_0,Gender_1,ID,...,Var2_0,Var2_1,Var2_2,Var2_3,Var2_4,Var2_5,Var2_6,Var4,Var5,target
0,37,0,1,1,0.0,1,0,1,0,ID000002C20,...,0,0,0,0,0,0,1,1,0,0.0
1,30,0,1,0,0.0,1,0,0,1,ID000004E40,...,0,0,0,0,0,0,1,3,13,0.0


In [5]:
def splitData(data,features):
    X = data.loc[~data.target.isna(),features]
    Y = data[~data.target.isna()].target
    X_train, X_test, y_train, y_test = train_test_split( X, Y,test_size = 0.3, random_state = 100,stratify=Y)
    return X,Y,X_train,X_test,y_train,y_test

def initModelPara(data,target,IDcol):
    target = target
    IDcol = IDcol
    predictors = [x for x in data.columns if x not in [IDcol]]
    features = [x for x in data.columns if x not in [IDcol,target]]

    alg = lgb.LGBMClassifier(
     learning_rate =0.1,
     n_estimators=1000,
     max_depth=5,
     min_child_weight=1,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary',
     nthread=4,
     metric='auc',
     seed=27)
    return alg,predictors,features

In [7]:
import eli5
from eli5.sklearn import PermutationImportance

#Create a sample model to calculate which features are more important.
alg,predictors,features=initModelPara(data,'target','ID')
X,Y,X_train,X_test,y_train,y_test=splitData(data,features)
rfc_model = RandomForestClassifier(random_state=0).fit(X_train, y_train)
perm = PermutationImportance(rfc_model, random_state=1).fit(X_test, y_test)

In [8]:
eli5.show_weights(perm, feature_names = X_test.columns.tolist(),top=150)

Weight,Feature
0 ± 0.0000,Var1_15
0 ± 0.0000,Var1_4
0 ± 0.0000,Var1_16
0 ± 0.0000,Var2_5
0 ± 0.0000,Var1_17
0 ± 0.0000,Var2_3
0 ± 0.0000,Var1_18
0 ± 0.0000,Var1_12
0 ± 0.0000,Var2_0
0 ± 0.0000,Var1_14
